<a href="https://colab.research.google.com/github/msaif28/GenAI/blob/main/custom_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Dataset: 2023 Fashion Trends
To complete my custom chatbot project, I have chosen the 2023 Fashion Trends dataset. This dataset contains information about the latest fashion trends for the year 2023, including styles, popular colors, fabric types, and other key fashion details. The dataset contains 82 rows and 3 different columns, making it well-suited for this application.


### User Case Scenario
The custom chatbot is being developed to deliver timely and detailed insights on the fashion trends of 2023 to users keen on keeping abreast of the newest styles and designs. Through incorporating this dataset, the chatbot will have the capability to address precise inquiries regarding fashion trends, encompassing information on favored colors, materials, styles, and accessories for the year 2023.

This tailored functionality will prove advantageous for fashion aficionados, stylists, shoppers, and anyone looking to refresh their wardrobe with the season's latest trends. Moreover, it will act as a crucial resource for retailers, designers, and marketers within the fashion sector who require up-to-the-minute fashion intelligence to cater to their clients' needs effectively.

By providing reliable updates on 2023 fashion trends, the chatbot aims to aid users in making well-informed fashion decisions, elevating their style quotient, and maintaining a competitive edge in the ever-evolving fashion landscape.


### Data Wrangling



In [160]:
!pip install -r /home/saifm/custom_chatbot/requirements.txt -q


In [161]:
import pandas as pd
import numpy as np
import openai

In [162]:
df= pd.read_csv('/home/saifm/custom_chatbot/data/2023_fashion_trends.csv')

In [163]:
df.head()

,URL,Trends,Source
0,https://www.refinery29.com/en-us/fashion-trend...,2023 Fashion Trend: Red. Glossy red hues took ...,7 Fashion Trends That Will Take Over 2023 — Sh...
1,https://www.refinery29.com/en-us/fashion-trend...,2023 Fashion Trend: Cargo Pants. Utilitarian w...,7 Fashion Trends That Will Take Over 2023 — Sh...
2,https://www.refinery29.com/en-us/fashion-trend...,"2023 Fashion Trend: Sheer Clothing. ""Bare it a...",7 Fashion Trends That Will Take Over 2023 — Sh...
3,https://www.refinery29.com/en-us/fashion-trend...,2023 Fashion Trend: Denim Reimagined. From dou...,7 Fashion Trends That Will Take Over 2023 — Sh...
4,https://www.refinery29.com/en-us/fashion-trend...,2023 Fashion Trend: Shine For The Daytime. The...,7 Fashion Trends That Will Take Over 2023 — Sh...


In [164]:
df.shape

(82, 3)

In [165]:
df.Trends

0     2023 Fashion Trend: Red. Glossy red hues took ...
1     2023 Fashion Trend: Cargo Pants. Utilitarian w...
2     2023 Fashion Trend: Sheer Clothing. "Bare it a...
3     2023 Fashion Trend: Denim Reimagined. From dou...
4     2023 Fashion Trend: Shine For The Daytime. The...
                            ...                        
77    If lime green isn't your vibe, rest assured th...
78    "As someone who can clearly (not fondly) remem...
79    "Combine this design shift with the fact that ...
80    Thought party season ended at the stroke of mi...
81    "This season, we saw the revival of the bubble...
Name: Trends, Length: 82, dtype: object

In [166]:
## Checking every row of Trends has information

df[df["Trends"].str.len() == 0]

,URL,Trends,Source


In [167]:
## Since we working on Trends columns as per statement Other sources are not required so i am dropping this

df = df.drop(['URL', 'Source'], axis=1)


In [168]:
df.head()

,Trends
0,2023 Fashion Trend: Red. Glossy red hues took ...
1,2023 Fashion Trend: Cargo Pants. Utilitarian w...
2,"2023 Fashion Trend: Sheer Clothing. ""Bare it a..."
3,2023 Fashion Trend: Denim Reimagined. From dou...
4,2023 Fashion Trend: Shine For The Daytime. The...


In [169]:
df.columns = ['text' if x=='Trends' else x for x in df.columns]


In [170]:
df.head()

,text
0,2023 Fashion Trend: Red. Glossy red hues took ...
1,2023 Fashion Trend: Cargo Pants. Utilitarian w...
2,"2023 Fashion Trend: Sheer Clothing. ""Bare it a..."
3,2023 Fashion Trend: Denim Reimagined. From dou...
4,2023 Fashion Trend: Shine For The Daytime. The...


### Custom Query Completion

In [171]:
import openai


In [172]:
# Function to get embeddings in batches
def get_embeddings(dataframe, column_name, model_name, batch_size=100):
    """
    Fetch embeddings for a specified column in a pandas DataFrame using OpenAI's embedding model in batches.

    :param dataframe: pandas DataFrame containing the text data.
    :param column_name: The name of the column containing text to embed.
    :param model_name: The name of the OpenAI embedding model to use.
    :param batch_size: The number of rows to process in each batch.
    :return: List of embeddings.
    """
    embeddings = []
    for i in range(0, len(dataframe), batch_size):
        # Extract the batch of text
        batch_texts = dataframe.iloc[i:i+batch_size][column_name].tolist()
        # Fetch embeddings for the batch
        response = openai.Embedding.create(
            input=batch_texts,
            engine=model_name
        )
        # Extract embeddings from the response and add to the list
        batch_embeddings = [data["embedding"] for data in response["data"]]
        embeddings.extend(batch_embeddings)
    return embeddings

# Set your API key and embedding model name
openai.api_key = ""
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"

# Call the function to get embeddings
df['embeddings'] = get_embeddings(df, 'text', EMBEDDING_MODEL_NAME, 100)

# Display the head of the DataFrame to verify
df.head()

,text,embeddings
0,2023 Fashion Trend: Red. Glossy red hues took ...,"[-0.020933903753757477, -0.02226594276726246, ..."
1,2023 Fashion Trend: Cargo Pants. Utilitarian w...,"[-0.001787635963410139, -0.028945190832018852,..."
2,"2023 Fashion Trend: Sheer Clothing. ""Bare it a...","[-0.010452313348650932, -0.019246812909841537,..."
3,2023 Fashion Trend: Denim Reimagined. From dou...,"[-0.0155369583517313, -0.005411861930042505, 0..."
4,2023 Fashion Trend: Shine For The Daytime. The...,"[-0.004937830381095409, 0.0017760044429451227,..."


In [173]:
df.to_csv("/home/saifm/custom_chatbot/embeddings.csv")

In [174]:

# Assuming your embeddings are saved in a column named "embeddings" in the CSV file
csv_file_path = "/home/saifm/custom_chatbot/embeddings.csv"  # Specify the path to your CSV file

# Load the CSV file into a DataFrame
df = pd.read_csv(csv_file_path, index_col=0)

# Convert the string representations of the embeddings back into NumPy arrays
df['embeddings'] = df['embeddings'].apply(lambda x: np.array(eval(x)))

# Display the DataFrame to verify the embeddings are correctly loaded
df.head(50)


,text,embeddings
0,2023 Fashion Trend: Red. Glossy red hues took ...,"[-0.020933903753757477, -0.02226594276726246, ..."
1,2023 Fashion Trend: Cargo Pants. Utilitarian w...,"[-0.001787635963410139, -0.028945190832018852,..."
2,"2023 Fashion Trend: Sheer Clothing. ""Bare it a...","[-0.010452313348650932, -0.019246812909841537,..."
3,2023 Fashion Trend: Denim Reimagined. From dou...,"[-0.0155369583517313, -0.005411861930042505, 0..."
4,2023 Fashion Trend: Shine For The Daytime. The...,"[-0.004937830381095409, 0.0017760044429451227,..."
5,2023 Fashion Trend: Maxi Skirts. In response t...,"[-0.025447335094213486, -0.01921859383583069, ..."
6,2023 Fashion Trend: Cobalt Blue. The strongest...,"[-0.0069767870008945465, -0.021418340504169464..."
7,Sculptural Bags That Make a Simple Statement. ...,"[-0.027015026658773422, -0.0008984121959656477..."
8,All of the Pastels. Pastels are classic (albei...,"[-0.007083786651492119, -0.002227324992418289,..."
9,Denim-On-Denim. McKenna’s second most anticipa...,"[-0.018341973423957825, -0.015721691772341728,..."


In [175]:


from scipy.spatial.distance import cosine


def get_embeddings(texts, model_name):
    """
    Fetch embeddings for a list of texts using OpenAI's API.
    """
    response = openai.Embedding.create(
        input=texts,
        engine=model_name
    )
    return [item["embedding"] for item in response["data"]]

def cosine_similarity(vec1, vec2):
    """
    Compute the cosine similarity between two vectors.
    """
    # Note: Subtracting from 1 because cosine() gives distance, not similarity
    return 1 - cosine(vec1, vec2)

def get_rows_sorted_by_relevance_alternate(question, df, model_name):
    """
    Alternate function to sort dataframe rows based on relevance to a question.
    """
    # Get the embedding for the question
    question_embedding = get_embeddings([question], model_name)[0]

    # Calculate cosine similarity between question embedding and each row's embedding
    similarities = df["embeddings"].apply(lambda emb: cosine_similarity(question_embedding, emb))

    # Add similarities to the dataframe
    df_copy = df.copy()
    df_copy["similarities"] = similarities

    # Sort by similarities in descending order (higher similarity = more relevant)
    df_copy.sort_values("similarities", ascending=False, inplace=True)

    return df_copy



In [176]:
q1_prompt = """
Question: "What is new in fashion in 2023."
Answer:
"""

q1_answer = openai.Completion.create(
    model="gpt-3.5-turbo-instruct", # using laster instruct gpt model
    prompt=q1_prompt,
    max_tokens=150
)["choices"][0]["text"].strip()

print(q1_answer)


It is difficult to predict specific fashion trends for the year 2023 as the fashion industry is constantly evolving and changing. However, fashion forecasters and experts have identified some potential trends that may be popular in 2023:

1. Sustainable and ethical fashion: With increasing awareness about environmental and social issues, there has been a growing demand for sustainable and ethical fashion. In 2023, we may see more designers and brands using environmentally friendly materials and ethical production methods.

2. Genderless fashion: In recent years, there has been a blurring of traditional gender roles and fashion has reflected this shift. In 2023, we may see a greater focus on unisex and genderless clothing, with more androgynous styles and designs.


In [177]:
q2_prompt = """
Question: "what colour is the trends in fall 2023?"
Answer:
"""

q2_answer = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt=q2_prompt,
    max_tokens=150
)["choices"][0]["text"].strip()

print(q2_answer)

It is impossible to accurately predict the color trends for Fall 2023 at this time. Fashion trends are constantly changing and are heavily influenced by the designers, consumer preferences, and other factors that cannot be predicted so far in advance.


In [178]:
import tiktoken

def create_prompt(question, df, max_token_count):
    """
    Given a question and a dataframe containing rows of text and their
    embeddings, construct a text prompt for a Completion model.
    """
    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("cl100k_base")

    # Define the static parts of the prompt to pre-calculate their token counts
    prompt_template_start = """
    Answer the question based on the context below, and if the question
    can't be answered based on the context, say "I don't know"

    Context:"""

    prompt_template_end = """

    ---

    Question: {}

    Answer:"""

    # Calculate the token count for static parts of the prompt and the question
    static_token_count = len(tokenizer.encode(prompt_template_start)) + \
                         len(tokenizer.encode(prompt_template_end.format(question)))

    # Calculate the maximum token count available for the context
    max_context_token_count = max_token_count - static_token_count - len(tokenizer.encode(question))

    # Initialize variables to build and track context
    context = []
    context_token_count = 0

    # Iterate through rows sorted by relevance to add to the context
    for text in get_rows_sorted_by_relevance(question, df)["text"].values:
        text_token_count = len(tokenizer.encode(text))

        # Check if adding this text would exceed the max token count for context
        if context_token_count + text_token_count <= max_context_token_count:
            context.append(text)
            context_token_count += text_token_count
        else:
            break

    # Construct and return the full prompt
    full_context = "\n\n###\n\n".join(context)
    return f"{prompt_template_start}\n\n{full_context}{prompt_template_end.format(question)}"



In [179]:


def custom_query(question, df, max_prompt_tokens=3000, max_answer_tokens=750):
    """
    Given a question, a dataframe containing rows of text, and a maximum number of desired tokens
    in the prompt and response, returns the answer to the question according to an OpenAI Completion model.
    If the model encounters an error, it returns an empty string.
    """

    # Ensure you're using the correct model identifier here
    model_name = "gpt-3.5-turbo-instruct"  # Correct model identifier for GPT-3.5 Turbo

    # Generate the prompt using your prompt creation function
    prompt = create_prompt(question, df, max_prompt_tokens)

    try:
        # Use the OpenAI API to generate a completion based on the prompt
        response = openai.Completion.create(
            model=model_name,
            prompt=prompt,
            max_tokens=max_answer_tokens,
            stop=None  # Optional: Specify stopping criteria if needed
        )
        # Extract and return the trimmed response text
        return response.choices[0].text.strip()
    except Exception as e:
        # Optionally log the error message for debugging purposes
        print(f"Error generating completion: {e}")
        return ""


In [180]:
print(create_prompt("what was the best color in trend 2023?", df, 1000))


    Answer the question based on the context below, and if the question
    can't be answered based on the context, say "I don't know"

    Context:

2023 Fashion Trend: Cobalt Blue. The strongest color story to come out of Spring 2023 runways, cobalt blue has burst through the collections with the freshness of a sea mist on a morning day. Just bright enough to warrant a double take, yet subtle enough to be worked into daily wear, it's the type of deep blue that will excite even the most color-averse. Bonus points: It pairs well with Pantone's Viva Magenta.

###

2023 Fashion Trend: Red. Glossy red hues took over the Fall 2023 runways ranging from Sandy Liang and PatBo to Tory Burch and Wiederhoeft. Think: Juicy reds with vibrant orange undertones that would look just as good in head-to-toe looks (see: a pantsuit) as accent accessory pieces (shoes, handbags, jewelry).

###

"Every season, there is a trend that speaks to the inner maximalist in us. While autumn gave us lurid neons, bik

Question01

Custom Performance Demonstration

In [181]:
print(q1_answer)

It is difficult to predict specific fashion trends for the year 2023 as the fashion industry is constantly evolving and changing. However, fashion forecasters and experts have identified some potential trends that may be popular in 2023:

1. Sustainable and ethical fashion: With increasing awareness about environmental and social issues, there has been a growing demand for sustainable and ethical fashion. In 2023, we may see more designers and brands using environmentally friendly materials and ethical production methods.

2. Genderless fashion: In recent years, there has been a blurring of traditional gender roles and fashion has reflected this shift. In 2023, we may see a greater focus on unisex and genderless clothing, with more androgynous styles and designs.


In [182]:
Q1= "What is new in fashion in 2023"
result_custom_q1 = custom_query(Q1, df)

print(f"Custom query result for Question 1: {result_custom_q1}")

Custom query result for Question 1: There are several trends emerging in 2023 fashion, including sheer clothing, shine for daytime, red hues, cobalt blue, denim reimagined, maxi skirts, and cargo pants. Other trends include elevated basics, 3D floral motifs, utilitarian wear, and indie sleaze. Fashion is also seeing a resurgence of tailored silhouettes, white cotton, ballet shoes, and green hues. Other trends relate to trains, cinched waists, form-fitting prints, sheer fashion, bubble skirts, and leather jackets. Finally, new digitally manipulated prints are gaining popularity as well.


Question02

In [183]:
print(q2_answer)

It is impossible to accurately predict the color trends for Fall 2023 at this time. Fashion trends are constantly changing and are heavily influenced by the designers, consumer preferences, and other factors that cannot be predicted so far in advance.


In [184]:
Q2= "What is new in fashion in 2023"
result_custom_q2 = custom_query(Q2, df)

print(f"Custom query result for Question 2: {result_custom_q2}")

Custom query result for Question 2: Sheer clothing, shine for daytime, bold red hues, cobalt blue, denim reimagined, cargo pants, indie sleaze, maxi skirts, simplicity in everyday dressing, 3D floral embellishments, perfectly cut trousers, mesh, pinstripe tailoring, neo-minimalism, revealed nudity, ballet shoes, trains on dresses, cinched blazers, formfitting patterns, bubble skirts, and embellished leather jackets are all major trends for fashion in 2023.
